## Bring your own constraints!
The following example shows how your own constraints can be enforced on the input
variables.

In [1]:
from entmoot import Enting, ProblemConfig, GurobiOptimizer, PyomoOptimizer
from entmoot.benchmarks import (
    build_multi_obj_categorical_problem,
    eval_multi_obj_cat_testfunc,
)

This part is pretty standard.

In [2]:
# define problem
problem_config = ProblemConfig(rnd_seed=73)
# number of objectives
number_objectives = 2
build_multi_obj_categorical_problem(problem_config, n_obj=number_objectives)

# sample data
rnd_sample = problem_config.get_rnd_sample_list(num_samples=20)
testfunc_evals = eval_multi_obj_cat_testfunc(rnd_sample, n_obj=number_objectives)

params = {"unc_params": {"dist_metric": "l1", "acq_sense": "exploration"}}
enting = Enting(problem_config, params=params)
# fit tree ensemble
enting.fit(rnd_sample, testfunc_evals)

How to add constraints depends on wether you are using Gurobi or Pyomo

### Gurobi Version

In [4]:
# get optimization model
model_gur = problem_config.get_gurobi_model_core()
# extract decision variables
x = model_gur._all_feat[3]
y = model_gur._all_feat[4]
z = model_gur._all_feat[5]
# add constraint that all variables should coincide
model_gur.addConstr(x == y)
model_gur.addConstr(y == z)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2512524
Academic license 2512524 - for non-commercial use only - registered to t.___@imperial.ac.uk


<gurobi.Constr *Awaiting Model Update*>

It is important to update the Gurobi model. Otherwise the constraints will not be added to the model!

In [5]:
model_gur.update()

Now you can run the optimization and verify that the variable values indeed equal each other.

In [6]:
# Build GurobiOptimizer object and solve optimization problem
params_gurobi = {"MIPGap": 0}
opt_gur = GurobiOptimizer(problem_config, params=params_gurobi)

In [7]:
res_gur = opt_gur.solve(enting, model_core=model_gur)

Set parameter MIPGap to value 0
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2512524 - for non-commercial use only - registered to t.___@imperial.ac.uk
Optimize a model with 3172 rows, 1828 columns and 11438 nonzeros
Model fingerprint: 0xa6d043a3
Model has 100 SOS constraints
Variable types: 1803 continuous, 25 integer (24 binary)
Coefficient statistics:
  Matrix range     [5e-08, 2e+04]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 6e+00]
  RHS range        [2e-04, 8e+03]
Presolve removed 367 rows and 248 columns
Presolve time: 0.12s
Presolved: 2805 rows, 1580 columns, 9988 nonzeros
Presolved model has 94 SOS constraint(s)
Variable types: 1557 continuous, 23 integer (23 binary)

Root relaxation: unbounded, 647 iterations, 0.04 seconds

In [8]:
# Build GurobiOptimizer object and solve optimization problem
params_gurobi = {"MIPGap": 0}
opt_gur = GurobiOptimizer(problem_config, params=params_gurobi)

res_gur = opt_gur.solve(enting, model_core=model_gur)
x_opt, y_opt, z_opt = res_gur.opt_point[3:]

assert round(x_opt, 5) == round(y_opt, 5) and round(y_opt, 5) == round(z_opt, 5)

Set parameter MIPGap to value 0
Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Academic license 2512524 - for non-commercial use only - registered to t.___@imperial.ac.uk
Optimize a model with 3172 rows, 1828 columns and 11438 nonzeros
Model fingerprint: 0x7f777cf7
Model has 100 SOS constraints
Variable types: 1803 continuous, 25 integer (24 binary)
Coefficient statistics:
  Matrix range     [5e-08, 2e+04]
  Objective range  [1e+00, 2e+00]
  Bounds range     [1e+00, 6e+00]
  RHS range        [2e-04, 8e+03]
Presolve removed 367 rows and 248 columns
Presolve time: 0.07s
Presolved: 2805 rows, 1580 columns, 9984 nonzeros
Presolved model has 94 SOS constraint(s)
Variable types: 1557 continuous, 23 integer (23 binary)

Root relaxation: unbounded, 862 iterations, 0.06 seconds

In [14]:
al = opt_gur._active_leaves
# list[tuple[int, str]]


IndexError: list index out of range

### Pyomo Version

In [ ]:
# Pyomo version
import pyomo.environ as pyo

model_pyo = problem_config.get_pyomo_model_core()
# extract decision variables
x = model_pyo._all_feat[3]
y = model_pyo._all_feat[4]
z = model_pyo._all_feat[5]
# add constraint that all variables should coincide
model_pyo.xy_equal_constr = pyo.Constraint(expr=x == y)
model_pyo.yz_equal_constr = pyo.Constraint(expr=y == z)

# Build GurobiOptimizer object and solve optimization problem
params_pyomo = {"solver_name": "gurobi"}
opt_pyo = PyomoOptimizer(problem_config, params=params_pyomo)

res_pyo = opt_pyo.solve(enting, model_core=model_pyo)
x_opt, y_opt, z_opt = res_pyo.opt_point[3:]

assert round(x_opt, 5) == round(y_opt, 5) and round(y_opt, 5) == round(z_opt, 5)

Note that no model update is required in the Pyomo version in contrast to the Gurobi variant